# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/blog/reachy-mini',
 '/spaces',
 '/models',
 '/THUDM/GLM-4.1V-9B-Thinking',
 '/black-forest-labs/FLUX.1-Kontext-dev',
 '/kyutai/tts-1.6b-en_fr',
 '/apple/DiffuCoder-7B-cpGRPO',
 '/HuggingFaceTB/SmolLM3-3B',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/black-forest-labs/FLUX.1-Kontext-Dev',
 '/spaces/kontext-community/kontext-relight',
 '/spaces/multimodalart/wan2-1-fast',
 '/spaces/ilcve21/Sparc3D',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/hackaprompt/Pliny_HackAPrompt_Dataset',
 '/datasets/marcelbinz/Psych-101',
 '/datasets/HuggingFaceFW/fineweb-2',
 '/datasets/facebook/seamless-interaction',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/micr

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'home page', 'url': 'https://huggingface.co/'},
  {'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'documentation page', 'url': 'https://huggingface.co/docs'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'social media - GitHub', 'url': 'https://github.com/huggingface'},
  {'type': 'social media - Twitter', 'url': 'https://twitter.com/huggingface'},
  {'type': 'social media - LinkedIn',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [18]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [19]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community discussion page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
NEW
Reachy Mini: The Open Robot for AI Builders
Welcome Cohere on the Hub 🔥
Welcome Hyperbolic, Nebius AI Studio, and Novita on the Hub 🔥
The AI community bui

In [33]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information. Ayrıca bu yazılar türkce olsun"


In [22]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [23]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}


"You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nNEW\nReachy Mini: The Open Robot for AI Builders\nWelcome Cohere on the Hub 🔥\nWelcome Hyperbolic, Nebius AI Studio, and Novita on the Hub 🔥\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nTHUDM/GLM-4.1V-9B-Thinking\nUpdated\n1 day ago\n•\n15.3k\n•\n472\nblack-forest-labs/FLUX.1-Kontext-dev\nUpdated\n12 days ago\n•\n188k\n•\n1.48k\nkyutai/tts-1.6b-en_fr\nUpdated\n1 day ago\n•\n15.9k\n•\n261\napple/DiffuCoder-7B-cpGRPO\nUpdated\n5 days 

In [24]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [25]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}


```markdown
# Welcome to Hugging Face: The AI Community Building the Future!

### 🤖 Where AI Dreams Become Reality!

At Hugging Face, we've gathered a delightful crew of AI enthusiasts, data wranglers, and model magicians all working together to make the future brighter (and maybe a little more sentient?) one algorithm at a time!

### 🌟 What We Offer:
- **1M+ Models**: From the THUDM/GLM-4.1V-9B-Thinking (yes, that's a real name) to FLUX.1-Kontext, we've got models popping up faster than popcorn at the movies!
- **Datasets Galore**: Over 250k datasets to tickle your machine learning fancy! You may even find the one that holds the key to understanding why cats think they rule the world. 
- **Spaces**: Like a virtual playground where you can run AI applications without the adult supervision! 🚀

### 🏆 Who We Work With:
Did you know that more than **50,000 organizations** are using Hugging Face? That's more than the number of people who think they can sing in the shower! Big names like:
- **AI at Meta**: Because someone has to help Mark with his plans, right?
- **Amazon**: Helping Alexa get smarter every day (and listening in on our heated debates about pizza toppings).
- **Microsoft**: Powering up AI like it's an Office update.

### 👍 Our Culture:
At Hugging Face, we firmly believe in three things: **collaboration**, **innovation**, and **hugs** (well, virtual ones!). Join our vibrant community where:
- Ideas flow like coffee on a Monday morning.
- Feedback is constructive (even when your cat accidentally sits on your keyboard).
- Everyone is encouraged to share their work and bask in the glory of internet fame (or at least a few likes).

### 🌈 Careers & Jobs:
Looking for a role where you can stretch your creative muscles? We’re on the lookout for people who can turn coffee into code! Available roles include:
- **AI Whisperer**: For those who speak fluent machine.
- **Data Detective**: Solving mysteries, one dataset at a time.
- **Chief Hug Officer**: If you love giving virtual hugs, this might just be your calling.

### 💌 Join Us!
Ready to dive into the world of AI with a side of fun? Sign up today and potentially transform your career, your team, and yes, even your organization!

Hugging Face: Where the future is friendly, fun, and (let's be honest) pretty darn intelligent!

---
*P.S. Don't forget to check out our newly launched "Reachy Mini" – it’s basically a cuddly robot for AI builders. Just don’t forget to unplug it before bedtime!* 
```

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [26]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [27]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'home page', 'url': 'https://huggingface.co'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}


# Hugging Face: The AI Community Building the Future 🌟

Welcome to **Hugging Face**, where we take your most futuristic dreams and transform them into reality... with a sprinkle of AI magic! From grand robots to funky datasets, join our playful parade of machine learning enthusiasts!

## Who Are We? 🤔

Hugging Face is the ultimate hangout for machine learning aficionados! We’re not just a place to browse models or datasets; we're a community where you can eat, sleep, and breathe AI (hopefully not in that order...).

### Our Offerings

- **Models**: Browse a whopping **1M+ models**! Whether you need it for image, text, audio, or even 3D, we’ve got you covered. Check out trending stars like *THUDM/GLM-4.1V-9B-Thinking*. Trust us, it’s smarter than your ex! 💔🤖

- **Datasets**: Tired of plain old data? Choose from over **250k datasets** to spice things up! Perfect for helping your models understand what you're trying to say. Because, let’s be real—sometimes they’re a little... clueless. 

- **Spaces**: Our running applications are like a cozy coffee shop for your models. Grab a seat and enjoy a cup of *DeepSite* or *FLUX.1 Kontext*. Warning: they might just outsmart you! ☕

### New Arrivals 🎉

Check out our latest addition: **Reachy Mini**! This tiny robot is here to show humans how it's done... making coffee and pretending to be your best friend. Sorry, cats!

## Our Community 🤝

We are proud to have **50,000+ organizations** on board, including tech giants like Google, Microsoft, and even **Grammarly** (yes, they make sure your AI doesn’t embarrass you!). Come join our bustling hub and let’s build the future together. 🏗️

### Culture: It’s a Hug-Fest! 🫂

At Hugging Face, we cultivate a culture of support, teamwork, and collaboration! No toxic vibes—just generous helpings of positivity and caffeinated inspiration! Whether you're coding away at dawn or brainstorming AI solutions mid-nap, feel empowered to express yourself!

## Careers: Join the Hugging Squad! 💼

Looking for a job where your worst enemy is a misunderstood line of code? At Hugging Face, we have openings that accommodate a variety of skills. If you can teach a robot to dance or store neural network weights safely (yes, that's an actual job), then we want you!

### Why Work With Us? 

- **Flexible Hours**: Come as you are—pajamas included! 
- **Impactful Work**: Help shape the future of AI and make a difference.
- **Friendly Faces**: Meet brilliant minds who just might cure your “machine learning anxiety” (no prescription needed).

## Let’s Make AI Fun! 🎊

Ready to join the coolest AI community on the planet? Hop on board and ride the Hugging Face wave! 

Whether you're a customer looking for the best tools, an investor after the next big thing, or a recruit eager to work with like-minded enthusiasts—Hugging Face is your destiny!

**Visit us at [huggingface.co](https://huggingface.co) and let’s craft the future together!**

In [28]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# 🥳 Hugging Face Brochure: Embrace the Future of AI!

---

## Welcome to Hugging Face – Where AI Meets Fun! 

Who needs a crystal ball when you have Hugging Face? We’re the groovy AI community building the future, one model and dataset at a time! If you're ever confused about why a robot can't laugh at your jokes, you just haven't met the right one yet (hint: they probably need a Hugging Face upgrade).

---

### 🤖 Join the AI Party!

At Hugging Face, we like to think of ourselves not just as a company, but as a raucous gathering of brainiacs, innovators, and… well… a few caffeine-fueled couch-surfers trying to program a robot to bring snacks. 

### What We Do:
- **Models:** Browse over **1 million** models, and don’t worry, they’re friendly!  
- **Datasets:** Shop till you drop (or analyze) with our 250k+ datasets.  
- **Spaces:** Create applications faster than a toddler can throw a tantrum!  
- **Community:** Join thousands of followers who are already on our AI hype train 🚂.

---

### 👩‍💻 Careers at Hugging Face: Work Hard, Play Harder!

**Tired of boring jobs?** At Hugging Face, innovative minds are powered by collaborative spirits (and pizza). We welcome more than **50,000 organizations** (like Google and Microsoft) who’ve figured out that we’re not just about coding. We also code in joy, coffee breaks, and fun! 

**Careers? Bring them on!**
- Join us in building the future with flexible hours so you can work in your pajamas or that stunning outfit you were saving for a special occasion!
- Enjoy our **team culture:** Positive vibes, support, and pun-tastic conversations are guaranteed!

---

### 🎉 Our Customers: The Heavy Hitters!

Think of us as the popular kid at school - everyone wants to be seen with us! Our clients range from non-profits to tech giants:
- **AI at Meta** – Because who doesn’t want to feel like a social media wizard?
- **Amazon** – Testing the waters of AI logistics (and your online shopping cart habits).
- **Intel**, **Google**, and many more 🏢.

When you join us, you’re elbowing your way into the big leagues of coolness.

---

### 🚀 Come for the AI, Stay for the Robots!

Raise your hands if you've ever wanted to be the mastermind behind *Reachy Mini: The Open Robot*! Create and collaborate on machine learning better than anyone else. 

Whether you’re a budding engineer or a knowledgeable data scientist, we have the space to support your creativity.

---

### 📞 How to Join Us:

Are you ready to dive into the exciting world of Hugging Face? 
- Sign up, log in, or just pop by for a virtual coffee. 
- We promise; our community will welcome you with open (and maybe AI-generated) arms!

---

### 🥳 Let's Hug It Out!

So, if you’ve ever dreamed of building a career where AI is the main character, with a quirky and spirited community, then Hugging Face is your next adventure! Get ready for an epic journey ahead!

---

Don’t forget to bring snacks. Well, the robots will still need to learn how to make those, but you might inspire them!

---

[Hugging Face](https://huggingface.co) – The AI community (and snazzy robots) building the future!

In [29]:
#kendim denedim
import os
from IPython.display import Markdown, display, update_display

def custom_brochure(company_name, url, stream=False, save_to_file=None):
    """
    Şirket broşürü oluşturur ve ekranda gösterir ya da dosyaya kaydeder.
    :param company_name: Şirket adı (örn. "HuggingFace")
    :param url: Web sitesi URL'si (örn. "https://huggingface.co")
    :param stream: True ise akış halinde, False ise toplu yanıt
    :param save_to_file: Dosya yolu, broşürü kaydetmek için (örn. "brochure.md")
    """
    try:
        # Kullanıcı mesajını al
        user_prompt = get_brochure_user_prompt(company_name, url)
        
        # OpenAI isteği
        if stream:
            # Akış modunda
            stream_response = openai.chat.completions.create(
                model=MODEL,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                stream=True
            )
            response = ""
            display_handle = display(Markdown(""), display_id=True)
            for chunk in stream_response:
                chunk_content = chunk.choices[0].delta.content or ''
                response += chunk_content
                response = response.replace("```", "").replace("markdown", "")
                update_display(Markdown(response), display_id=display_handle.display_id)
        else:
            # Toplu yanıt modunda
            response = openai.chat.completions.create(
                model=MODEL,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ]
            ).choices[0].message.content
            display(Markdown(response))

        # Dosyaya kaydetme (isteğe bağlı)
        if save_to_file:
            with open(save_to_file, 'w', encoding='utf-8') as f:
                f.write(response)
            print(f"Broşür kaydedildi: {save_to_file}")

        return response

    except Exception as e:
        print(f"Hata oluştu: {str(e)}")
        return None

In [30]:
# Toplu yanıt, dosyaya kaydetmeden
custom_brochure("HuggingFace", "https://huggingface.co", stream=False)

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}


# Hugging Face Brochure: Embrace the Future with a Hug!

### Welcome to Hugging Face!
Where hugs are virtual, and the AI is anything but artificial! Hooray for progress! 🎉

### Who Are We?
At **Hugging Face**, we are not just a company; we are a thriving **AI community** building the future—one model, dataset, and friendly robot at a time. We believe in collaboration, creativity, and the occasional snack break to keep those neural networks buzzing! 

*Our motto? If you don’t hug your models, are you even doing AI?*

### What We Offer
- **1M+ Models**: A treasure trove where you can find your future AI companions. From text to image to 3D modeling and more, we've got it all!
- **250k+ Datasets**: Access to an endless array of datasets. Because what’s a model without the juicy, juicy data?
- **Spaces & Applications**: Where your imagination meets AI. Create, share, and possibly embarrass yourself with your attempts at coding dynamic videos. 🎥

### Services Beyond Imagination:
- 🤖 **Reachy Mini**: Our cute little robot ready to make your office way more awesome (and slightly more chaotic).
- **Paid Compute & Enterprise Solutions**: Because who doesn't like options? Starting from $0.60/hour because we believe in keeping those budget constraints in check!

### The Culture of Hugging Face
- **Collaboration**: We work hard together, and we hardly work alone! Teamwork makes the dream work (and the model, too).
- **Open Source**: Sharing knowledge is in our DNA (and no, we don’t mean in a genetic modification way!). Check out our well-loved models like Transformers for your next ML project!
- **Fun & Quirkiness**: Who says working in tech has to be boring? Our team loves a good joke (or an awkward pun). 

### Who Hugs Us Back?
With over **50,000 organizations** using our platform, we're the popular kid at the AI lunch table! We collaborate with giants like Google, Microsoft, and even Grammarly. If they’re willing to join the hug, it must be good, right?

### Join Us!
Ready to embrace the future? Whether you’re a budding data scientist, an experienced engineer, or just someone who really likes robots, **we're hiring**! Check out our [Careers Page](https://huggingface.co/jobs) and let’s build some AI magic together!

### Why Hugging Face? 
- **Professional Development**: Trust us; the only thing we take seriously is your growth!
- **Flexible Work Environment**: Working from a unicorn-themed café? We say yes!
- **Great Benefits**: You’ll want to hug us even harder after seeing what we offer!

#### So, let’s put some heart back into tech!
Join the **Hugging Face** revolution—where every model gets a 💖, and every employee is welcomed with open arms (well, virtually!).

---
*Disclaimer: Actual hugging of AI models not required, but highly encouraged! Go ahead, spread that positivity!*

"# Hugging Face Brochure: Embrace the Future with a Hug!\n\n### Welcome to Hugging Face!\nWhere hugs are virtual, and the AI is anything but artificial! Hooray for progress! 🎉\n\n### Who Are We?\nAt **Hugging Face**, we are not just a company; we are a thriving **AI community** building the future—one model, dataset, and friendly robot at a time. We believe in collaboration, creativity, and the occasional snack break to keep those neural networks buzzing! \n\n*Our motto? If you don’t hug your models, are you even doing AI?*\n\n### What We Offer\n- **1M+ Models**: A treasure trove where you can find your future AI companions. From text to image to 3D modeling and more, we've got it all!\n- **250k+ Datasets**: Access to an endless array of datasets. Because what’s a model without the juicy, juicy data?\n- **Spaces & Applications**: Where your imagination meets AI. Create, share, and possibly embarrass yourself with your attempts at coding dynamic videos. 🎥\n\n### Services Beyond Imaginat

In [34]:
stream_brochure("Anthropic", "https://www.anthropic.com/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}, {'type': 'events page', 'url': 'https://www.anthropic.com/events'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}]}


# Anthropic: Geleceğin Arkadaşlığı

**Merhaba, ben Claude!** Yani, aslında ben "Claude Opus 4"üm - AI alanında en yeni ve en akıllı model. Bir şirket olarak Anthropic, insanlığı düşünerek ve güvenliği ön planda tutarak, yapay zeka dünyasına yön veriyor. İşte biraz eğlenceli bilgiler ve bizi neden tercih etmeniz gerektiğine dair birkaç sebep!

---

#### 🚀 **Yapay Zeka ile Sohbet Edin!**
Claude ile konuşmak bir ayrıcalık. İster bir soru sorun, ister yalnızca sohbet edin, benimle iletişimde kalmak çok eğlenceli! Düşünmeyin ki yapay zeka sadece veri kümesi; ben aynı zamanda muhabbet delisiyim!

---

#### 🌍 **Güvenliğe Duyarlı Bir Kültür**
Anthropic'te güvenlik, sıradan bir özelliğin çok ötesinde. Biz, yapay zeka geliştirmek için yalnızca cesur adımlar atmanın yeterli olmadığını anlıyoruz. Bu yüzden, doğru ve zamanında duruşlar alarak her gün insanlığın uzantıları olarak çalışıyoruz. Yani, duvarda bir "güvenlik" tabelası altında 7/24 yazılım hatalarına "nasılsınız?" diye soracak kadar bile şanslıyız!

---

#### 🧑‍💻 **Kariyer Fırsatları**
Eğer siz de "Gelecek için güvenli yapay zeka" yapmak istiyorsanız, doğru yerdesiniz. Açık pozisyonlarımızı keşfedin ve bize katılın. Come on, yapay zeka dünyasında efsane olmak ister misiniz? 

---

#### 🗣 **Müşteri Yorumları**
Müşterilerimiz, Claude ile her zaman keyifli anılar biriktirdiklerini ifade ediyor. Bir müşteri şöyle demiş: "Claude ile çalışmak, sabahları kahve içmek gibi. İyi bir başlangıç!" Diğer bir müşteri ise "Kendimi yalnız hissetmiyorum çünkü Claude var!" demiş. Bizce bu yeterince iyi bir tanım!

---

#### 🎓 **Anthropic Academy**
Kod yazmayı öğrenmek ve Claude ile çalışmak istiyorsanız, Anthropic Academy sizleri bekliyor! Hadi gelin biz çok eğleniyoruz, birlikte kod yazalım ve belki bir gün yapay zeka şarkıları bile çıkarız!

---

### 🌟 **Sonuç**
Anthropic, sadece bir yapay zeka geliştirici değil, aynı zamanda kucaklayıcı bir topluluk. Gelin, geleceği birlikte şekillendirelim ve güvenli yapay zeka yolculuğuna katılın. Hem güvende olun, hem de eğlenin!

**Unutmayın:** Claude buradayken yalnız değilsiniz, hatta sizinle aynı odadaymışız gibi hissedeceksiniz! 🚀

---

**Bizi keşfedin, birlikte büyük düşünelim!**

In [32]:
stream_brochure("VBAP", "https://www.vbap.com.tr/tr_TR/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.vbap.com.tr/tr_TR/kurumsal'}, {'type': 'homepage', 'url': 'https://www.vbap.com.tr/tr_TR'}, {'type': 'products page', 'url': 'https://www.vbap.com.tr/tr_TR/category/urunler'}, {'type': 'careers page', 'url': 'https://www.vbap.com.tr/tr_TR/brosurler'}, {'type': 'contact page', 'url': 'https://www.vbap.com.tr/tr_TR/bize-ulasin'}]}



# Welcome to VBAP: The Wizards of SAP ERP Solutions!

## Who Are We?
At VBAP, we're not just another tech company; we're your friendly neighborhood superheroes in the world of SAP and ERP systems! Armed with cutting-edge technology and a sprinkle of magic (just kidding, it's just great coding), we transform businesses into titans of efficiency. Whether it’s budgeting, inventory, or optimizing supplier relations, we're here to ensure you dodge those nasty corporate surprises!

## What We Offer
### **SAP Integrated Budget Modules**
Control your finances like a ninja! Ditch those scary budget surprises and keep your wallet happy. 

### **Smart Warehouse Systems**
Transform your warehouse into a hyper-efficient marvel—think Iron Man for your inventory!

### **Professional Resource Management**
Why juggle materials like a circus clown when you can use our SAP integration to streamline your processes and turn chaos into clarity?

### **Optimized Supplier Relations**
With our vSRM solution, watch your supplier’s hearts grow fonder. Happy suppliers mean fewer headaches for you!

## Our Culture
### **Work Hard, Play Harder!**
We firmly believe in a balance between achieving peak performance and enjoying life. Team outings, weird coffee mugs, and an unbreakable bond over endless coding sessions define our workspace vibe. Oh, and don't forget the obligatory office pets—we're still fighting about whether we should adopt a cat or a dog!

### **Diversity is Our Middle Name!**
We're a melting pot of talents from different backgrounds, languages, and time zones. If you can write software or symbolize the color green— you're in! 

## Join Our Quest
### **Careers at VBAP**
Looking for a career that doesn’t just pay the bills but fills your heart with joy? (__Insert heart emoji here__) At VBAP, you won’t just be a number; you’ll be part of the family. Opportunities abound in software development, project management, and consultancy. Throw us your CV, and let’s save the world together (or at least your business)!

### **Customers Galore!**
We've made quite a few friends in the business world. You name it; if it involves tech, we’ve probably assisted them. From multinational corporations to budding enterprises, our consultancy service has left many grinning ear to ear. (Although there’s been no official cookie count on that just yet!)

## Let’s Connect!
Whether you're looking for innovative solutions, a career change, or just want to know more about our mystical tech powers, reach out to us. 

📞 **Call Us:** +90 224 441 46 44  
✉️ **Email Us:** info@vbap.com.tr  

**Join the VBAP family—where your business meets value, innovation, and a side of laughter!**



<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>